In [77]:
import warnings
warnings.filterwarnings("ignore")

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, Row, ArrayType, StringType
import pandas as pd
import json
import datetime
import re

master = 'local'
appName = 'Free time exercise'

spark = SparkSession.builder.appName(appName).getOrCreate()

if spark.sparkContext:
    print('===============')
    print(f'AppName: {spark.sparkContext.appName}')
    print(f'Master: {spark.sparkContext.master}')
    print('===============')
else:
    print('Could not initialise pyspark session')
    
# raw_data = spark.read.option("multiline", "true").json("file:///home/rahulw/PycharmProjects/Final_Project/Ingestion/Airflow_data_files/sample_part2.json")
df = spark.read.csv("file:///home/user1/Pendrive/Data_ingestion/Data_to_be_ingested/batch_2_DL.csv",header=True)
# raw_data.columns



AppName: Free time exercise
Master: local[*]


In [2]:
# df = raw_data.sample(0.001,True)

# df.createOrReplaceTempView("table1")

In [4]:
import pandas as pd
pd_df = df.toPandas()

pd_df['pd_date'] = pd.to_datetime(pd_df['DOB'])
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669323 entries, 0 to 669322
Data columns (total 24 columns):
 #   Column                                                           Non-Null Count   Dtype         
---  ------                                                           --------------   -----         
 0   ID                                                               669323 non-null  object        
 1   Name of DL Holder                                                669323 non-null  object        
 2   D.L. No.                                                         669323 non-null  object        
 3   DOB                                                              669323 non-null  object        
 4   Gender                                                           669323 non-null  object        
 5   Permanent Address                                                669323 non-null  object        
 6   Temporary Address                                                265

In [5]:
pd_df.head(3)

,ID,Name of DL Holder,D.L. No.,DOB,Gender,Permanent Address,Temporary Address,Email ID,Mobile,Blood Group,...,Bio-ID,Concatenated name and parts,Qualification code,Dependent relationship with the applicant,Applicant relative name and parts,Present village/District code/Present location type,Temporary village or town/District code/temporary location type,Permanent temporary sub district code,State code,pd_date
0,1,Alita Kumari,AP10 20198496335,02/07/1992,2,"T 147, 4 Floor, Paramjit City, Plot 37, Udipti...","The White House, Sahid Marg, Sector Number- 25...",alita@zohomail.com,7349665864,B-,...,None,None,0,F,Vinay Dhamale,None,None,None,AP,1992-02-07
1,3,Balvan Bharose Holla,KA21 20037324125,13/10/1978,1,"E/129, 2 Floor, Saroz Nagar, Lalasa Lane, Behi...",None,balvanholla@protonmail.com,8707498696,A+,...,None,None,8,F,Asiwani Holla,None,None,None,KA,1978-10-13
2,4,Komali De Soogoor,WB21 20189103941,22/07/1994,0,"P 146, 4 Floor, Thuvaradran Complex, Plot 17, ...","Tamanna, Agnikumara Lane, Nr. Sangeeta Market,...",komali.soogoor1994@hotmail.com,7614314222,AB+,...,None,None,10,O,Adhishri Soogoor,None,None,None,WB,1994-07-22


In [16]:
# pd_df["new_dob_1"] = pd.to_datetime(pd_df["DOB"], format='%d%m%Y')

In [15]:
from datetime import datetime
datetime.strptime('02/07/1992', '%m/%d/%Y')

datetime.datetime(1992, 2, 7, 0, 0)

In [12]:
# spark.sql("select * from table1").show(2)

In [14]:
# df.show(2)

In [6]:
df2 = spark.table("table1")

sorted(df.collect()) == sorted(df2.collect())

True

In [29]:
df.columns

['ID',
 'Name of DL Holder',
 'D.L. No.',
 'DOB',
 'Gender',
 'Permanent Address',
 'Temporary Address',
 'Email ID',
 'Mobile',
 'Blood Group',
 'Identification marks',
 'Place of birth',
 'Country citizenship',
 'NPR No.',
 'Bio-ID',
 'Concatenated name and parts',
 'Qualification code',
 'Dependent relationship with the applicant',
 'Applicant relative name and parts',
 'Present village/District code/Present location type',
 'Temporary village or town/District code/temporary location type',
 'Permanent temporary sub district code',
 'State code']

In [15]:
path = "file:///home/user1/Pendrive/Data_ingestion/Data_to_be_ingested/batch_2_DL.csv"

In [19]:
# df3 = spark.read.csv(path, schema=None, sep=None, encoding=None, quote=None, escape=None, comment=None, header=True, inferSchema=None, ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True, nullValue=False, nanValue=False, positiveInf=None, negativeInf=None, dateFormat=None, timestampFormat=None, maxColumns=None, maxCharsPerColumn=None, maxMalformedLogPerPartition=None, mode=None, columnNameOfCorruptRecord=None, multiLine=None, charToEscapeQuoteEscaping=None, samplingRatio=None, enforceSchema=None, emptyValue=None, locale=None, lineSep=None, pathGlobFilter=None, recursiveFileLookup=None, modifiedBefore=None, modifiedAfter=None, unescapedQuoteHandling=None)

In [21]:
import dateutil.parser
def change_date_format(arguments):
    try:
        if arguments:
            yourdate = dateutil.parser.parse(arguments)
            oldformat = yourdate
            datetimeobject = datetime.strptime(str(oldformat),'%Y-%m-%d  %H:%M:%S')
            newformat = datetimeobject.strftime('%d-%m-%Y')
            return newformat
        else:
            pass
    except Exception as e:
        pass

In [27]:
from pyspark.sql.functions import *
date_udf = udf(change_date_format)
df = df.withColumn('New_Date_of_Birth', date_udf(df.DOB))

In [34]:
from pyspark.sql.types import DateType
# df.select(col("DOB"), date_format(col("DOB"), "MM-dd-yyyy").alias("new_dob_format")).show()
df = df.withColumn("new_dob",df.DOB.cast(DateType()))

In [39]:
# df.select("new_dob").show()

In [42]:
pd_df['pd_date'] = pd.to_datetime(pd_df['DOB'])
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 26 columns):
 #   Column                                                           Non-Null Count  Dtype         
---  ------                                                           --------------  -----         
 0   ID                                                               650 non-null    object        
 1   Name of DL Holder                                                650 non-null    object        
 2   D.L. No.                                                         650 non-null    object        
 3   DOB                                                              650 non-null    object        
 4   Gender                                                           650 non-null    object        
 5   Permanent Address                                                650 non-null    object        
 6   Temporary Address                                                251 non-null    o

In [23]:
import dateutil.parser
def change_date_format(arguments):
    try:
        if arguments:
            yourdate = dateutil.parser.parse(arguments)
            oldformat = yourdate
            datetimeobject = datetime.strptime(str(oldformat),'%Y-%m-%d  %H:%M:%S')
            newformat = datetimeobject.strftime('%d-%m-%Y')
            return newformat
        else:
            pass
    except Exception as e:
        pass

In [30]:
from pyspark.sql.functions import udf
date_udf = udf(change_date_format)
df = df.withColumn('New_Date_of_Birth', date_udf(df.DOB))

In [31]:
df.select("DOB","New_Date_of_Birth").show()

+----------+-----------------+
|       DOB|New_Date_of_Birth|
+----------+-----------------+
|02/07/1992|       07-02-1992|
|13/10/1978|       13-10-1978|
|22/07/1994|       22-07-1994|
|24/09/1983|       24-09-1983|
|09/06/1982|       06-09-1982|
|12/09/1975|       09-12-1975|
|04/03/1991|       03-04-1991|
|27/12/2002|       27-12-2002|
|16/02/1991|       16-02-1991|
|03/07/2000|       07-03-2000|
|02/11/1975|       11-02-1975|
|22/05/2001|       22-05-2001|
|26/03/1982|       26-03-1982|
|22/04/1999|       22-04-1999|
|15/03/1994|       15-03-1994|
|29/10/1990|       29-10-1990|
|20/10/1986|       20-10-1986|
|29/07/1983|       29-07-1983|
|16/01/2002|       16-01-2002|
|01/08/1994|       08-01-1994|
+----------+-----------------+
only showing top 20 rows



In [86]:
import sys
import os

In [89]:
table = 'aaa'
csv_path = '/home/user1/Downloads/' + table + '.csv'
os.system("touch "+csv_path)

0

In [39]:
# os.system("sudo apt install mdbtools")

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper


256

In [40]:
!pip install meza

     |████████████████████████████████| 56 kB 633 kB/s eta 0:00:01
     |████████████████████████████████| 103 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 630 kB 896 kB/s eta 0:00:01
     |████████████████████████████████| 133 kB 1.2 MB/s eta 0:00:01
  Created wheel for ijson: filename=ijson-2.6.1-py3-none-any.whl size=17443 sha256=673eacf775454dc9f524c1d1ad125e2058ca3b854cc147e77b3c671010451814
  Stored in directory: /home/user1/.cache/pip/wheels/d9/bf/75/d521a020ef274e6e95a67d5cc167bd26a485a72b296ac06c20
  Created wheel for python-slugify: filename=python_slugify-1.2.6-py2.py3-none-any.whl size=4609 sha256=3f3abcd0587f71b8be06478b6655ae227181fddabc80d303c84230307cf5d320
  Stored in directory: /home/user1/.cache/pip/wheels/f6/93/e2/71ef4b74b771520f05ba206cb557a4b04d017dc0ccb84606fe
Successfully built ijson python-slugify
  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1
  

In [56]:
from meza import io
records = io.read("/home/user1/Downloads/campaign_template1/Campaign_Template.mdb")

# print(next(records))
rec = io.read_mdb("/home/user1/Downloads/campaign_template1/Campaign_Template.mdb")


SyntaxError: invalid syntax (3037432022.py, line 5)

In [57]:
!pip install mdb_parser

  Created wheel for mdb-parser: filename=mdb_parser-0.0.3-py3-none-any.whl size=2277 sha256=8c6981903f5172c7549a3d1aaf9a7a3dc25fa159a30dc3af93f310e810f78fb2
  Stored in directory: /home/user1/.cache/pip/wheels/21/ea/8b/a56b63662291d32f4460e5fb6a70808de0b1fb9d2266f96775
Successfully built mdb-parser


In [82]:
template = "/home/user1/Downloads/campaign_template1/Campaign_Template.mdb"

In [65]:
sample = "/home/user1/Downloads/sample.mdb"

In [ ]:
import os
from mdb_parser import MDBParser, MDBTable
tables= []
def mdb_to_txt(mdb_file):
    db = MDBParser(file_path=mdb_file)
    tables = db.tables
    for table in tables:
        

### Converting mdb file's tables into csv format

In [97]:
import os
from mdb_parser import MDBParser, MDBTable
tables= []
dir_path = "/home/user1/Downloads/mdb_files/"
def mdb_dir(dir_path):
    x = os.system("ls " + dir_path)
    # lst = list(x)
    print(x)

mdb_dir(dir_path)

sample  2.mdb
sample  3.mdb
sample.mdb
0


In [138]:
!pip install pypyodbc

  Created wheel for pypyodbc: filename=pypyodbc-1.3.6-py3-none-any.whl size=22873 sha256=43828f70db307b3742a9fbabd1efc62c10834878441a7716d100f5b01417f15d
  Stored in directory: /home/user1/.cache/pip/wheels/4a/b0/00/a014827d8966a4b2601f6bff537634a1fe3e7b4269fe668879
Successfully built pypyodbc


In [140]:
import pypyodbc as py

In [91]:
import os
from mdb_parser import MDBParser, MDBTable
tables= []

def mdb_to_txt(mdb_file):
    db = MDBParser(file_path=mdb_file)
    tables = db.tables
            
    for table in tables:
        
        table=table.replace(' ','_')
        
        csv_path = '/home/user1/Downloads/' + table + '1.csv'
        os.system("touch "+ csv_path)
        
        f = open(csv_path, 'w')        
        
        parsed_table = MDBTable(file_path=mdb_file, table=table)
        # create the csv writer
        writer = csv.writer(f)
        
        for row in parsed_table:
            # write a row to the csv file
            writer.writerow(row)
            
    # close the file
    f.close()
mdb_to_txt(sample)


In [ ]:
import csv

# open the file in the write mode
f = open('path/to/csv_file', 'w')

# create the csv writer
writer = csv.writer(f)

# write a row to the csv file
writer.writerow(row)

# close the file
f.close()


In [94]:
from mdb_parser import MDBParser, MDBTable
# path = "/home/user1/Downloads/campaign_template1/Campaign_Template.mdb"
path = "/home/user1/Downloads/sample.mdb"

db = MDBParser(file_path=path)

# tables = db.tables
# Get and print the database tables
tables = db.tables

for table in tables:
    table=table.replace('_','__')
    table = MDBTable(file_path=path, table=table)
    print(table.columns)
    for row in table:
        print(row)


['ID', 'Description', 'Type']
['1', 'FYIcenter.com', 'FYI']
['2', 'Active Server Pages', 'ASP']
['3', 'JavaServer Pages', 'JSP']
['4', 'Portable Network Graphics', 'PNG']
['5', 'Graphics Interchange Format', 'GIF']
['6', ' Windows Media Video', 'WMV']


#### creating list of files in directory

In [98]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [121]:
dirName = "/home/user1/Downloads/mdb_files/";
# Get the list of all files in directory tree at given path
listOfFiles = getListOfFiles(dirName)
print(listOfFiles)

['/home/user1/Downloads/mdb_files/sample.mdb', '/home/user1/Downloads/mdb_files/sample3.mdb', '/home/user1/Downloads/mdb_files/sample2.mdb']


In [131]:
NewlistOfFiles = []
for i in listOfFiles:
    NewlistOfFiles.append(i.replace(" ","_"))
print(NewlistOfFiles)

['/home/user1/Downloads/mdb_files/sample.mdb', '/home/user1/Downloads/mdb_files/sample3.mdb', '/home/user1/Downloads/mdb_files/sample2.mdb']


In [135]:
import os
from mdb_parser import MDBParser, MDBTable
tables= []

def mdb_to_txt(mdb_file):
    mdb_file=mdb_file.replace(' ','_')
    print(mdb_file)
    db = MDBParser(file_path=mdb_file)
    
    tables = db.tables
            
    for table in tables:
        
        table=table.replace(' ','_')
        
        csv_path = '/home/user1/Downloads/' + table + '1.csv'
        os.system("touch "+ csv_path)
        
        f = open(csv_path, 'w')        
        
        parsed_table = MDBTable(file_path=mdb_file, table=table)
        # create the csv writer
        writer = csv.writer(f)
        
        for row in parsed_table:
            # write a row to the csv file
            writer.writerow(row)
            
    # close the file
    print(f"{csv_path} created \n")
    f.close()
    
    
# sample = "/home/user1/Downloads/sample.mdb"
# mdb_to_txt(sample)

In [136]:
tables= []
for file in NewlistOfFiles:
    mdb_to_txt(file)

/home/user1/Downloads/mdb_files/sample.mdb
/home/user1/Downloads/Sample1.csv created 

/home/user1/Downloads/mdb_files/sample3.mdb
/home/user1/Downloads/Sample1.csv created 

/home/user1/Downloads/mdb_files/sample2.mdb
/home/user1/Downloads/Sample1.csv created 



### Pan Type extraction from pan number

In [6]:
pan_dict = {"P":"Person", "C":"Company"}


In [63]:
name = "KJUpJ23145"

def pan_func(args):
    if len(args.strip()) == 10:
        char = args[3].upper()
        print(char)
        if char in pan_dict.keys():
            pan_type = pan_dict[char]
            return pan_type
    return "Unknown"
    
pan_func(name)   

P


'Person'

### dl regex

In [78]:
# dl_regex = "(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s][A-Z]?[0-9]{2}[-/\s](19|20)[0-9]{9}"

In [4]:
import re

dl_regex = "(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[0-9]{2}[A-Z]?(19|20)[0-9]{9}"
clean_dl_regex = "[^a-zA-Z0-9]"

my_dl = "MH26@#$%A^20190009319"

clean_dl = re.sub(clean_dl_regex, r"",my_dl)
print(clean_dl)

regex = re.compile(dl_regex)
if regex.search(clean_dl):
    print("match")
else:
    print("no match")

MH26A20190009319
match


In [28]:
dl_regex = r"\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]?/?[A-Z]{3}/?[0-9]{2}/?[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?[0-9]{2}[-/\s]?[A-Z]?[-/\s]?[0-9]{4}[-/\s]?[0-9]+|[0-9]{4}[-/\s]?[A-Z]{2}[-/\s]?[0-9]{2}[-/\s]?[A-Z][-/\s]?[0-9]{4}|\b[0-9]+(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[0-9]+|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)o[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]([0-9]{2})?[-/\s]?[0-9]+\b"

In [29]:
dl_regex

'\\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\\s]?([0-9]{2})?[-/\\s]?/?[A-Z]{3}/?[0-9]{2}/?[0-9]+\\b|\\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\\s]?[0-9]{2}[-/\\s]?[A-Z]?[-/\\s]?[0-9]{4}[-/\\s]?[0-9]+|[0-9]{4}[-/\\s]?[A-Z]{2}[-/\\s]?[0-9]{2}[-/\\s]?[A-Z][-/\\s]?[0-9]{4}|\\b[0-9]+(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[0-9]+|\\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)o[0-9]+\\b|\\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\\s]?([0-9]{2})?[-/\\s]([0-9]{2})?[-/\\s]?[0-9]+\\b'

In [32]:
import re
def driving_licence_validation(arguments):
    if arguments is not None:
        driving_licence = re.sub('[\\?,$;#:”/.*@!&]','',arguments)
        # print(driving_licence)
        # if len(driving_licence) > 1 and driving_licence[2] != '-':
            # driving_licence1 = ' '.join([driving_licence[i:i+4] for i in range(0,4,4)])
            # driving_licence = driving_licence1+" "+"".join([driving_licence[i] for i in range(4,len(driving_licence))])
        print(driving_licence)
        # valid_dl = re.search('^(([A-Z]{2}[0-9]{2})( )|([A-Z]{2}-[0-9]{2}))((19|20)[0-9][0-9])[0-9]{7}$', driving_licence)
        valid_dl = re.search(dl_regex,driving_licence)
        if valid_dl != None:
            return driving_licence
        else:
            return driving_licence + " invalid driving licence number"


In [33]:
print(driving_licence_validation("MH-26-2019-@@@0009319"))

MH-26-2019-0009319
MH-26-2019-0009319


### phone validation

In [76]:
import phonenumbers
def phoneNumValidation(phone):
    try:
        my_num = phonenumbers.parse(phone)
        print(my_num)
        c = phonenumbers.country_code_for_region(my_num)
        # print(my_num,c)
    except:
        return phone

In [77]:
phoneNumValidation("+918177992436")

Country Code: 91 National Number: 8177992436


'+918177992436'

In [78]:
phone_regex = "^[6-9]{1}[0-9]{9}$"
all_punc = "[^0-9]"
def phone_val(original_phone):
    try:
        phone = re.sub(all_punc,r"",original_phone)
        if len(str(phone)) <=12:
            clean_mob = phone[-10:]
            if re.search(phone_regex,clean_mob):
                return clean_mob + " valid"
            return original_phone + " invalid"
        return original_phone + " invalid"
    except:
        return original_phone + " invalid"          

In [79]:
phone_val("8959506886")

'8959506886 valid'

In [80]:
phone_val("+08177992430")

'8177992430 valid'

In [81]:
phone_val("98177992436")

'8177992436 valid'

### Gender validation

In [45]:
#pip install difflib

In [47]:
dl_regex = "\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]?/?[A-Z]{3}/?[0-9]{2}/?[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?[0-9]{2}[-/\s]?[A-Z]?[-/\s]?[0-9]{4}[-/\s]?[0-9]+|[0-9]{4}[-/\s]?[A-Z]{2}[-/\s]?[0-9]{2}[-/\s]?[A-Z][-/\s]?[0-9]{4}|\b[0-9]+(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[0-9]+|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)o[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]([0-9]{2})?[-/\s]?[0-9]+\b"

In [6]:
import re
def driving_licence_validation(arguments):
    if arguments is not None:
        driving_licence = re.sub(r'[\\?,$;#:”/.*@!&]',r'',arguments)
        # print(driving_licence)
        # if len(driving_licence) > 1 and driving_licence[2] != '-':
            # driving_licence1 = ' '.join([driving_licence[i:i+4] for i in range(0,4,4)])
            # driving_licence = driving_licence1+" "+"".join([driving_licence[i] for i in range(4,len(driving_licence))])
        print(driving_licence)
        # valid_dl = re.search('^(([A-Z]{2}[0-9]{2})( )|([A-Z]{2}-[0-9]{2}))((19|20)[0-9][0-9])[0-9]{7}$', driving_licence)
        valid_dl = re.search(dl_regex,driving_licence)
        if valid_dl != None:
            return driving_licence
        else:
            return driving_licence+" "+"invalid driving licence number"


In [54]:
print(driving_licence_validation("TN220180001521"))

TN220180001521
TN220180001521 invalid driving licence number


In [7]:
def driving_licence_validation(arguments):
    if arguments is not None:
        driving_licence = re.sub(r'[\\\s?,$;#:”/.*@!&]',r'',arguments)
        if len(driving_licence) > 1 and driving_licence[2] != '-':
            driving_licence1 = ' '.join([driving_licence[i:i+4] for i in range(0,4,4)])
            driving_licence = driving_licence1+" "+"".join([driving_licence[i] for i in range(4,len(driving_licence))])
        valid_dl = re.search("\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]?/?[A-Z]{3}/?[0-9]{2}/?[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?[0-9]{2}[-/\s]?[A-Z]?[-/\s]?[0-9]{4}[-/\s]?[0-9]+|[0-9]{4}[-/\s]?[A-Z]{2}[-/\s]?[0-9]{2}[-/\s]?[A-Z][-/\s]?[0-9]{4}|\b[0-9]+(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[0-9]+|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)o[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]([0-9]{2})?[-/\s]?[0-9]+\b", driving_licence)
        if valid_dl != None:
            return driving_licence
        else:
            return driving_licence+" "+"invalid driving licence number"

In [8]:
driving_licence_validation("TN50Z20100002570")

'TN50 Z20100002570 invalid driving licence number'

In [88]:
dl_regex = r"\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]?/?[A-Z]{3}/?[0-9]{2}/?[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?[0-9]{2}[-/\s]?[A-Z]?[-/\s]?[0-9]{4}[-/\s]?[0-9]+|[0-9]{4}[-/\s]?[A-Z]{2}[-/\s]?[0-9]{2}[-/\s]?[A-Z][-/\s]?[0-9]{4}|\b[0-9]+(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[0-9]+|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)o[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]([0-9]{2})?[-/\s]?[0-9]+\b"

In [89]:
print(dl_regex)

\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]?/?[A-Z]{3}/?[0-9]{2}/?[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?[0-9]{2}[-/\s]?[A-Z]?[-/\s]?[0-9]{4}[-/\s]?[0-9]+|[0-9]{4}[-/\s]?[A-Z]{2}[-/\s]?[0-9]{2}[-/\s]?[A-Z][-/\s]?[0-9]{4}|\b[0-9]+(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[0-9]+|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)o[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]([0-9]{2})?[-/\s]?[0-9]+\b


In [90]:
if re.search(dl_regex,"TN50Z20100002570"):
    print("match")
else:
    print("no match")

match


In [91]:
print(re.search(dl_regex,"MH1220191234567"))

<re.Match object; span=(0, 15), match='MH1220191234567'>


In [73]:
len("TN50Z20100002570")

16

In [74]:
len("MH1220191234567")

15

In [1]:
def driving_licence_validation(arguments):
    if arguments is not None:
        driving_licence = re.sub(r'[\\\s?,$;#:”/.*@!&]',r'',arguments)
        if len(driving_licence) > 1 and driving_licence[2] != '-':
            for i in range(0,4,4):
                driving_licence1 = ' '.join([driving_licence[i:i+4]]) 
            for i in range(4,len(driving_licence)):
                driving_licence = driving_licence1+" "+"".join([driving_licence[i]
        valid_dl = re.search("\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?([0-9]{2})?[-/\s]?/?[A-Z]{3}/?[0-9]{2}/?[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[-/\s]?[0-9]{2}[-/\s]?[A-Z]?[-/\s]?[0-9]{4}[-/\s]?[0-9]+|[0-9]{4}[-/\s]?[A-Z]{2}[-/\s]?[0-9]{2}[-/\s]?[A-Z][-/\s]?[0-9]{4}|\b[0-9]+(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[0-9]+|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)o[0-9]+\b|\b(AN|AP|AR|AS|BR|CG|CH|DN|DD|DL|GA|GJ|HR|HP|JK|JH|KA|KL|LA|LD|MP|MH|MN|ML|MZ|NL|OD|OR|PY|PB|RJ|SK|TN|TS|TR|UP|UK|WB)[/\s]?([0-9]{2})?[-/\s]?([0-9]{2})?[-/\s]?[0-9]+\b", driving_licence)
        if valid_dl != None:
            return driving_licence
        else:
            return driving_licence+" "+"invalid driving licence number"

SyntaxError: invalid syntax (329369669.py, line 9)

In [68]:
import re
num_punc = "[^a-zA-z\\s.]"
def clean_name(name):
    if name is not None:
        clean_name = re.sub(num_punc,r"",name.strip().lower())
        if len(clean_name)==0:
            return None
        elif clean_name[1]=="." and clean_name[3]==".":
            return clean_aname.replace("."," ").title()
        elif clean_name.split(".",1)[0] in salutations:
            only_name = clean_name.split(".",1)[1]
            return only_name.replace("."," ").title()
        else:
            return clean_name.replace("."," ").title()

In [72]:
clean_name("         ")

In [73]:
clean_name("Rahul @@Wangawar")

'Rahul Wangawar'

In [71]:
clean_name(None)